In [147]:
import numpy as np
import pandas as pd
import scipy
from scipy.spatial import distance
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from scipy.special import rel_entr
import math
from scipy.stats import wasserstein_distance

In [148]:
Predicted = pd.read_csv("./Seurat/Outputs/output_Sim13.csv")
Target=pd.read_csv("./Data/Sim13/target.csv")

In [149]:
Predicted.drop(columns="Unnamed: 0", inplace=True)
#Predicted.drop(columns="cell_ID", inplace=True)
Predicted.drop(columns="prediction.score.max",inplace=True)


In [150]:
Target.drop(columns="Unnamed: 0", inplace=True)

In [151]:
for i in range(len(Target)):
    Target.loc[i,:]=Target.loc[i,:]/Target.loc[i,:].sum()

In [152]:
for i in range(len(Predicted)):
    Predicted.loc[i,:]=Predicted.loc[i,:]/Predicted.loc[i,:].sum()
 

In [153]:
columns_p=[]
for i in range(Predicted.shape[1]):
    columns_p.append(Predicted.columns[i])
    

In [154]:
columns_t=[]
for i in range(Target.shape[1]):
    columns_t.append(Target.columns[i])
    

In [155]:
columns_p=sorted(columns_p)
columns_t=sorted(columns_t)

In [156]:
Predicted=Predicted[columns_p]
Target=Target[columns_t]

In [157]:
Target=Target.to_numpy()
Predicted=Predicted.to_numpy()

## Pearson Correlation

In [158]:
num_celltypes=Predicted.shape[1]
d = np.zeros(num_celltypes)
for i in range(0,num_celltypes):
    p=Predicted[:,i]
    t=Target[:,i]
    prs=(np.corrcoef(p,t,rowvar=False))
    d[i]=prs[0,1]

/opt/homebrew/Caskroom/miniforge/base/envs/ML/lib/python3.10/site-packages/numpy/lib/function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/opt/homebrew/Caskroom/miniforge/base/envs/ML/lib/python3.10/site-packages/numpy/lib/function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [159]:
np.mean(np.nan_to_num(d))

0.5607457301927996

In [160]:
d

array([0.81008899, 0.09100758, 0.660544  , 0.50098726, 0.63852872,
       0.78314786, 0.64773071,        nan, 0.82090724, 0.65451495])

## Jenson Shannon Divergence

In [161]:
jns=distance.jensenshannon(Predicted,Target,axis=1)
jns=jns[~np.isinf(jns).squeeze()]
np.mean(jns)

0.46251257342139157

## F1 Score, Precision, Recall

In [162]:
thresholds={0.001,0.01,0.1,0.2,0.3}

In [163]:
d = pd.DataFrame(0, index=np.arange(len(thresholds)), columns=["Precision","Recall","F1-score"])
d.index=thresholds
for t in thresholds:
    Predicted_logical=(Predicted>t).astype(int)
    Target_logical=(Target>t).astype(int)
    d.loc[t,"Precision"]=precision_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"Recall"]=recall_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"F1-score"]=f1_score(Target_logical,Predicted_logical,average="micro")

In [164]:
d

,Precision,Recall,F1-score
0.100,0.760247,0.471941,0.582366
0.200,0.731844,0.476147,0.576934
0.300,0.659450,0.505424,0.572254
0.010,0.642023,0.599208,0.619877
0.001,0.632466,0.681743,0.656180


## Wasserstein Distance

In [165]:
scipy.stats.wasserstein_distance(np.reshape(Target,(Target.shape[0]*Target.shape[1])),np.reshape(Predicted,(Predicted.shape[0]*Predicted.shape[1])))

0.05120904956020148

## KL Divergence

In [166]:
to_be_kept=(Predicted>0)

In [167]:
Predicted=Predicted[to_be_kept]
Target=Target[to_be_kept]

In [168]:
kl=scipy.special.rel_entr(Target,Predicted)

In [169]:
np.mean(kl)

0.17358575907970375